In [1]:
import pandas as pd
from sqlalchemy import create_engine, text

In [2]:
df = pd.read_csv('main_customer_shopping_behavior.csv')

In [3]:
engine = create_engine(
    "mysql+pymysql://root:@localhost/retail_analytics"
)

In [4]:
df.head(10)

,customer_id,age,gender,item_purchased,category,purchase_amount,location,size,color,season,review_rating,subscription_status,shipping_type,discount_applied,previous_purchases,payment_method,frequency_of_purchases,age_group,days_of_frequency
0,1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Express,Yes,14,Venmo,Fortnightly,Middle-aged,14
1,2,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,3.1,Yes,Express,Yes,2,Cash,Fortnightly,Young Adult,14
2,3,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,3.1,Yes,Free Shipping,Yes,23,Credit Card,Weekly,Middle-aged,7
3,4,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,3.5,Yes,Next Day Air,Yes,49,PayPal,Weekly,Young Adult,7
4,5,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,2.7,Yes,Free Shipping,Yes,31,PayPal,Annually,Middle-aged,365
5,6,46,Male,Sneakers,Footwear,20,Wyoming,M,White,Summer,2.9,Yes,Standard,Yes,14,Venmo,Weekly,Middle-aged,7
6,7,63,Male,Shirt,Clothing,85,Montana,M,Gray,Fall,3.2,Yes,Free Shipping,Yes,49,Cash,Quarterly,Senior,90
7,8,27,Male,Shorts,Clothing,34,Louisiana,L,Charcoal,Winter,3.2,Yes,Free Shipping,Yes,19,Credit Card,Weekly,Young Adult,7
8,9,26,Male,Coat,Outerwear,97,West Virginia,L,Silver,Summer,2.6,Yes,Express,Yes,8,Venmo,Annually,Young Adult,365
9,10,57,Male,Handbag,Accessories,31,Missouri,M,Pink,Spring,4.8,Yes,2-Day Shipping,Yes,4,Cash,Quarterly,Middle-aged,90


#### Question 1
#### What is the total revenue generated by male vs. female customers?

In [16]:
pd.read_sql("""SELECT `gender`, SUM(purchase_amount) as Revenue
FROM `customer_shopping_analysis` 
GROUP BY `gender`""", engine)

,gender,Revenue
0,Female,75191.0
1,Male,157890.0


#### Question 2
#### Which customers used a discount but still spent more than the average purchase amount?

In [13]:
pd.read_sql("""SELECT * 
FROM `customer_shopping_analysis` 
WHERE purchase_amount > (SELECT AVG(purchase_amount) FROM `customer_shopping_analysis`) AND discount_applied = "Yes"
ORDER BY purchase_amount;
""", engine)

,customer_id,age,gender,item_purchased,category,purchase_amount,location,size,color,season,review_rating,subscription_status,shipping_type,discount_applied,previous_purchases,payment_method,frequency_of_purchases,age_group,days_of_frequency
0,40,70,Male,Pants,Clothing,60,Arizona,S,Turquoise,Summer,4.2,Yes,Express,Yes,18,Credit Card,Monthly,Senior,30
1,166,42,Male,Handbag,Accessories,60,Kentucky,S,Yellow,Winter,2.8,Yes,Store Pickup,Yes,7,Cash,Monthly,Adult,30
2,304,20,Male,Sandals,Footwear,60,Oregon,M,Turquoise,Summer,3.3,Yes,Next Day Air,Yes,49,Venmo,Weekly,Young Adult,7
3,534,57,Male,Sandals,Footwear,60,North Carolina,L,Pink,Spring,4.5,Yes,Next Day Air,Yes,17,Debit Card,Weekly,Middle-aged,7
4,558,18,Male,Shirt,Clothing,60,Alaska,M,White,Spring,2.9,Yes,Free Shipping,Yes,31,Debit Card,Weekly,Young Adult,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
834,1413,25,Male,Shorts,Clothing,100,Michigan,M,Charcoal,Summer,2.6,No,2-Day Shipping,Yes,43,Venmo,Annually,Young Adult,365
835,1422,68,Male,Dress,Clothing,100,Pennsylvania,S,Gold,Spring,3.0,No,Standard,Yes,24,Bank Transfer,Fortnightly,Senior,14
836,1457,59,Male,Coat,Outerwear,100,Montana,S,Olive,Spring,2.7,No,Store Pickup,Yes,30,PayPal,Every 3 Months,Senior,90
837,1480,48,Male,Coat,Outerwear,100,Washington,L,Black,Spring,4.0,No,Standard,Yes,2,Bank Transfer,Weekly,Middle-aged,7


#### Question 3
#### Which product category has the highest average review rating?

In [15]:
pd.read_sql("""SELECT item_purchased, ROUND(AVG(review_rating), 2) AS average_product_rating
FROM `customer_shopping_analysis` 
GROUP BY item_purchased 
ORDER BY average_product_rating DESC LIMIT 5;""", engine)

,item_purchased,average_product_rating
0,Gloves,3.86
1,Sandals,3.84
2,Boots,3.82
3,Hat,3.80
4,Handbag,3.78


#### Question 4
#### What is the most preferred shipping type among subscribed customers?

In [16]:
pd.read_sql("""SELECT shipping_type, COUNT(*) AS most_prefered
FROM `customer_shopping_analysis` 
WHERE `subscription_status` = "Yes"
GROUP BY shipping_type
ORDER BY most_prefered DESC""", engine)

,shipping_type,most_prefered
0,Express,190
1,Store Pickup,190
2,Free Shipping,177
3,Standard,175
4,Next Day Air,168
5,2-Day Shipping,153


#### Question 5
#### Do subscribed customers spend more?
#### Compare average spend and total revenue between subscribers and non-subscribers.

In [17]:
pd.read_sql("""SELECT subscription_status, 
COUNT(customer_id) AS total_customers,
AVG(purchase_amount) as average_spend, 
SUM(purchase_amount) as total_revenue 
FROM `customer_shopping_analysis`
GROUP BY subscription_status""", engine)

,subscription_status,total_customers,average_spend,total_revenue
0,No,2847,59.8651,170436.0
1,Yes,1053,59.4919,62645.0


#### Question 6
#### Which 5 products have the highest percentage of purchases with discounts applied?

In [18]:
pd.read_sql("""SELECT
    item_purchased,
    ROUND(SUM(CASE WHEN discount_applied = 'Yes' THEN 1 ELSE 0 END) 
        / COUNT(*) * 100, 2) AS discount_percentage
FROM customer_shopping_analysis
GROUP BY item_purchased
ORDER BY discount_percentage DESC
LIMIT 5;""", engine)

,item_purchased,discount_percentage
0,Hat,50.00
1,Sneakers,49.66
2,Coat,49.07
3,Sweater,48.17
4,Pants,47.37


#### Out of the total discounted revenue, how much percentage does each product contribute?
#### Calculate the percentage contribution of each item_purchased to the total purchase amount, considering only purchases where discount_applied = 'Yes'.

In [21]:
pd.read_sql("""SELECT item_purchased,
    ROUND(
        SUM(purchase_amount) 
        / (
            SELECT SUM(purchase_amount)
            FROM customer_shopping_analysis
            WHERE discount_applied = 'Yes'
        ) * 100,
    2) AS discounted_revenue_percentage
FROM customer_shopping_analysis
WHERE discount_applied = 'Yes'
GROUP BY item_purchased
ORDER BY discounted_revenue_percentage DESC;""",engine)

,item_purchased,discounted_revenue_percentage
0,Pants,4.87
1,Sweater,4.64
2,Hat,4.62
3,Coat,4.50
4,Dress,4.39
5,Shirt,4.38
6,Sneakers,4.35
7,Shorts,4.27
8,Belt,4.26
9,Boots,4.23


#### Question 7
#### Segment customers into New, Returning, and Loyal based on their total number of previous purchases, and show the count of each segment.

In [19]:
pd.read_sql("""SELECT
    customer_segment,
    COUNT(*) AS customer_count
FROM (
    SELECT
        customer_id,
        CASE
            WHEN previous_purchases = 1 THEN 'New'
            WHEN previous_purchases BETWEEN 2 AND 10 THEN 'Returning'
            ELSE 'Loyal'
        END AS customer_segment
    FROM customer_shopping_analysis
) t
GROUP BY customer_segment;
""",engine)

,customer_segment,customer_count
0,Loyal,3116
1,New,83
2,Returning,701


#### Question 8
#### Which payment method is most commonly used by customers who purchase frequently?

In [7]:
pd.read_sql("""SELECT payment_method, COUNT(payment_method) as usage_count
FROM `customer_shopping_analysis` 
WHERE days_of_frequency <= 14
GROUP BY payment_method
ORDER BY usage_count DESC""", engine)

,payment_method,usage_count
0,Cash,292
1,Credit Card,277
2,PayPal,274
3,Debit Card,269
4,Venmo,261
5,Bank Transfer,255


#### Question 9
#### Which season generates the highest total revenue?

In [8]:
pd.read_sql("""SELECT season, SUM(purchase_amount) AS total_revenue
FROM `customer_shopping_analysis` 
GROUP BY season
ORDER BY total_revenue DESC;""",engine)

,season,total_revenue
0,Fall,60018.0
1,Spring,58679.0
2,Winter,58607.0
3,Summer,55777.0


#### Question 10
#### Which customer age group contributes the most to total revenue?

In [24]:
pd.read_sql("""SELECT age_group, SUM(purchase_amount) AS total_revenue
FROM `customer_shopping_analysis` 
GROUP BY age_group
ORDER BY total_revenue DESC;""", engine)

,age_group,total_revenue
0,Young Adult,62143.0
1,Middle-aged,59197.0
2,Adult,55978.0
3,Senior,55763.0


#### Question 11
#### what are the top 3 most purchased products within each category

In [20]:
pd.read_sql("""WITH item_counts AS (
    SELECT category, item_purchased,
        COUNT(customer_id) AS total_orders,
        ROW_NUMBER() OVER (
            PARTITION BY category
            ORDER BY COUNT(customer_id) DESC) AS item_rank
    FROM customer_shopping_analysis
    GROUP BY category, item_purchased
)

SELECT item_rank, category, item_purchased, total_orders
FROM item_counts
WHERE item_rank <= 3
ORDER BY category, item_rank;""", engine)

,item_rank,category,item_purchased,total_orders
0,1,Accessories,Jewelry,171
1,2,Accessories,Sunglasses,161
2,3,Accessories,Belt,161
3,1,Clothing,Blouse,171
4,2,Clothing,Pants,171
5,3,Clothing,Shirt,169
6,1,Footwear,Sandals,160
7,2,Footwear,Shoes,150
8,3,Footwear,Sneakers,145
9,1,Outerwear,Jacket,163


#### Question 12
#### Are customers who are repeat buyers (more than 5 previous purchases) also likely to subscribe?

In [21]:
pd.read_sql("""SELECT
    subscription_status,
    COUNT(customer_id) AS repeat_buyers
FROM customer_shopping_analysis
WHERE previous_purchases > 5
GROUP BY subscription_status;
""", engine)

,subscription_status,repeat_buyers
0,No,2518
1,Yes,958
